## Бонус - как включить проверку pep8 в ipynb

In [1]:
# обновить ipython до версии >= 6.3, если еще не
!pip install --upgrade ipython

     |████████████████████████████████| 788kB 805kB/s eta 0:00:01
  Found existing installation: ipython 7.8.0
    Uninstalling ipython-7.8.0:
      Successfully uninstalled ipython-7.8.0


In [2]:
# установить нужные библиотеки
!pip install flake8 pycodestyle_magic

     |████████████████████████████████| 71kB 313kB/s eta 0:00:011


In [3]:
# загрузить функцию проверки кода
%load_ext pycodestyle_magic

In [4]:
# включить проверку кода
%pycodestyle_on 
# или
# %flake8_on 

In [5]:
# на код не по пеп8 вылезут предупреждения

a=1

3:2: E225 missing whitespace around operator


+ Нейминг оно не проверяет.
+ Репозиторий pycodestyle_magic https://github.com/mattijn/pycodestyle_magic
+ Подробнее про разницу между pycodstyle и flake8 и вобще про линтеры можно почитать тут http://books.agiliq.com/projects/essential-python-tools/en/latest/linters.html

# Mock

Модуль mock позволяет нам подменить объекты в коде на моки (макеты с заранее определенным поведением) на время тестов. Это позволяет упростить написание тестов и добиться полного контроля над поведением программы. 

In [6]:
import unittest
from unittest.mock import patch

## Patch

+ заменить один объект дургим на время тестов можно с помощью функции ***patch***
+ используется как-то так
```
patch('module.object.method', ...)
```
+ может быть так же в виде декоратора - аргументы будут те же
```
@patch('module.object.method', ...)
def test_something(...):
    ...
```
+ первый аргумент - путь до объекта/метода который надо заменить (через точки, так же как мы импортируем объекты)
+ следующие аргументы определяют на что и как именно заменить
+ подробнее про аргументы: https://docs.python.org/3/library/unittest.mock.html#patch   


**ВАЖНО - где именно заменять объект**
+ То есть какой именно путь писать в ***patch*** первым аргументом?
+ Основное правило - заменять объект нужно **там где он используется**, а не там откуда его импортировали. 
+ То есть если в модуле (***my_beautiful_module.py***), который мы хотим протестировать импортируется какой-то объект который мы хотим заменить на мок (в данном случае функция ***some_fucntion***)
```
from some_module import some_function
def my_function():
    result = some_method() + 1
    return result
```
+ То в тестах нужно делать вот так, (а не *'some_module.some_function'*)
```
@patch('my_beautiful_module.some_function', ...)
def test_my_function(...):
    ...
```
+ Довольно часто все работает, даже если делать неправильно, но далеко не всегда.
+ Подробнее про это: https://docs.python.org/3/library/unittest.mock.html#id6


## Тестируем пользовательский ввод

In [7]:
def answer():
    answer = input('Answer yes or no')
    if answer == 'yes':
        return 1
    return 0


# декоратор @patch
# обращаем внимание на то, что в функцию test_answer
# нужно добавить еще один параметр, тк декоратор patch
# создает мок-объект и передает его в функцию в качестве аргумента
class TestAnswer(unittest.TestCase):
    # у нас и тестируемый код и тесты в одном файле,
    # поэтому в качестве имени модуля __main__
    @patch('__main__.input', return_value='yes')
    def test_answer_yes(self, fake_input):
        result = answer()
        self.assertEqual(result, 1)

    @patch('__main__.input', return_value='no')
    def test_answer_no(self, fake_input):
        result = answer()
        self.assertEqual(result, 0)


# # альтернативный вариант - контекстный менеджер и функция patch
# # здесь параметр добавлять не надо
# class TestAnswer(unittest.TestCase):
#     def test_answer_yes(self):
#         with patch('__main__.input', return_value='yes'):
#             result = answer()
#             self.assertEqual(result, 1)

#     def test_answer_no(self):
#         with patch('__main__.input', return_value='no'):
#             result = answer()
#             self.assertEqual(result, 0)


# запустить все тесты
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

..
----------------------------------------------------------------------
Ran 2 tests in 0.003s

OK


## side_effect

 Используем параметр side_effect. 
 Туда можно передать много чего: 
 + любой итерируемый объект
 + функцию, которая будет вызвана с переданными в исходную функцию парамерами вместо нее
 + или исключение (тогда оно будет поднято в процессе выполнения теста)       
 
Подробнее в документации https://docs.python.org/3/library/unittest.mock.html#unittest.mock.Mock.side_effect


В данном случае он пригодится нам, т.к. функция, которую мы хотим подменить вызывается в цикле и нужно передать не одно возвращаемое значение, а их последовательность.

In [8]:
def answer():
    answer = 'no'
    while answer == 'no':
        answer = input('Answer yes or no')
        if answer == 'yes':
            return 1
        print('Please say yes')


class TestAnswer(unittest.TestCase):
    # в качестве инпута вернется 2 раза 'no', затем 'yes'
    @patch('__main__.input', side_effect=['no', 'no', 'yes'])
    def test_answer(self, fake_input):
        result = answer()
        self.assertEqual(result, 1)

# # альтернативный вариант - контекстный менеджер
# class TestAnswer(unittest.TestCase):
#     def test_answer(self):
#         with patch('__main__.input', side_effect=['no', 'no', 'yes']):
#             result = answer()
#             self.assertEqual(result, 1)


# запустить все тесты
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

.

Please say yes
Please say yes



----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


## Если нужно заменить на моки несколько объектов

В идеале тут стоит еще протестировать вывод - что печатается то, что нужно и нужное количество раз. 

In [9]:
import io


def answer():
    answer = 'no'
    while answer == 'no':
        answer = input('Answer yes or no')
        if answer == 'yes':
            return 1
        print('Please say yes')


class TestAnswer(unittest.TestCase):
    # подменяем стандартный вывод (sys.stdout) на мок, из котрого потом
    # можно будет достать все, что напечаталось
    # помним о порядке применения декораторов - начиная с ближнего к функции
    # поэтому у нас fake_input первым аргументом, fake_output - вторым
    @patch('sys.stdout', new_callable=io.StringIO)
    @patch('__main__.input', side_effect=['no', 'no', 'yes'])
    def test_answer(self, fake_input, fake_output):
        result = answer()
        outputs = fake_output.getvalue().strip().split('\n')
        self.assertEqual(result, 1)
        self.assertEqual(len(outputs), 2)
        self.assertEqual(['Please say yes', 'Please say yes'], outputs)


# class TestAnswer(unittest.TestCase):
#     def test_answer(self):
#         # записываем мок заменяющий sys.stdout в переменную fake_output,
#         # чтобы потом к нему обращаться
#         with patch('__main__.input', side_effect=['no', 'no', 'yes']), \
#           patch('sys.stdout', new=io.StringIO()) as fake_output:
#             result = answer()
#             outputs = fake_output.getvalue().strip().split('\n')
#             self.assertEqual(result, 1)
#             self.assertEqual(len(outputs), 2)
#             self.assertEqual(['Please say yes', 'Please say yes'], outputs)


# запустить все тесты
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


## Read-only properties

+ Используем декоратор @property и защищенный/приватный атрибут, в зависимости от того, насколько мы не доверяем пользователю

In [10]:
class MyClass:
    def __init__(self, num):
        self._my_property = num

    @property
    def my_property(self):
        return self._my_property


my_obj = MyClass(1)
print(my_obj.my_property)
my_obj.my_property = 10

1


AttributeError: can't set attribute

Если атрибут имеет изменяемый тип данных 

In [ ]:
class MyClass:
    def __init__(self, num_list):
        self._my_list_property = num_list

    @property
    def my_property(self):
        return self._my_list_property


my_obj = MyClass([1, 2, 3])

Если сделать как в предыдущем варианте, список все еще можно будет изменить с помощью append или присваиванием по индексу элемента

In [ ]:
my_obj.my_property.append(10)
print(my_obj.my_property)

In [ ]:
my_obj.my_property[0] = 10
print(my_obj.my_property)

Чтобы так не было нужно сделать так, чтобы доступный для чтения атрибут (***my_property***) не ссылылся на исходный изменяемый объект (***__my_list_property***).      
Можно возвращать копию исходного объекта:

In [ ]:
class MyClass:
    def __init__(self, num_list):
        self._my_list_property = num_list

    @property
    def my_property(self):
        return self._my_list_property.copy()


my_obj = MyClass([1, 2, 3])

+ Append и изменение элемента происходят в копии (поэтому все работает, а не выбрасывает исключение).
+ Исходный список (тот, на который ссылается ***__my_list_property*** остается неизменным.
+ При обращении к атрибуту ***my_property*** возвращается исходный список, который остался неизменным.

In [ ]:
my_obj.my_property.append(10)
print(my_obj.my_property)

In [ ]:
my_obj.my_property[0] = 10
print(my_obj.my_property)

Можно сделать исходный объект неизменяемым и возвращать список созданный из него: 

In [ ]:
class MyClass:
    def __init__(self, num_list):
        self._my_list_property = tuple(num_list)

    @property
    def my_property(self):
        return list(self._my_list_property)


my_obj = MyClass([1, 2, 3])

In [ ]:
my_obj.my_property.append(10)
print(my_obj.my_property)

In [ ]:
my_obj.my_property[0] = 10
print(my_obj.my_property)